In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from mnist import MNIST

In [2]:
mnist_loader = MNIST("../data/mnist/")
mnist_loader.gz = True

In [3]:
images, labels = mnist_loader.load_training()

In [4]:
timages, tlabels = mnist_loader.load_testing()

In [5]:
images = np.concatenate((np.array(images), np.array(timages)))
labels = np.concatenate((np.array(labels), np.array(tlabels)))

In [6]:
images.shape

(70000, 784)

In [7]:
del timages
del tlabels

# batching

In [8]:
n_rows = images.shape[0]
batch_size = 2500
path = '../data/mnist/cos_sim.memmap'

In [9]:
fp = np.memmap(path, dtype='float64', mode='w+', shape=(n_rows, n_rows))

In [10]:
from tqdm import tqdm

In [11]:
%%time
for batch_index in tqdm(range(n_rows // batch_size)):
    start = batch_size * batch_index
    end = batch_size * (batch_index + 1)
    fp[start:end] = cosine_similarity(X=images[start:end], Y=images)

100%|██████████| 28/28 [12:41<00:00, 27.19s/it]

CPU times: user 5min 10s, sys: 27.5 s, total: 5min 37s
Wall time: 12min 41s


In [12]:
del fp